In [1]:
import re
import pandas as pd

# Import Dataset

In [2]:
dataset = pd.read_csv('data/train.csv')
dataset.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


# Preprocessing

## Preprocessing functions

In [3]:
def drop_ticket(dataset):
    dataset = dataset.drop('Ticket', axis=1)
    return dataset

In [4]:
def get_all_family_members(dataset):
    dataset['FamilyMembers'] = dataset['SibSp'] + dataset['Parch']
    return dataset

In [5]:
def clean_nans(dataset):
    dataset['Cabin'] = dataset['Cabin'].map(lambda x: x[0] if isinstance(x, str) else 'Z')
    dataset['Embarked'] = dataset['Embarked'].fillna('None')
    dataset['Age'] = dataset['Age'].fillna(-100)
    
    return dataset

In [6]:
def extract_title(name):
    match = re.search(r',\s*([^\.]*)\.', name)
    if match:
        return match.group(1).strip()
    return ""

In [7]:
def transform_name(name):
    maiden_name_match = re.search(r'\(([^()]*\b\w+)\)', name)
    last_name_match = re.match(r'([^,]+),', name)
    title_match = re.search(r',\s*([^\.]*)\.', name)

    last_name = last_name_match.group(1).strip()
    
    if title_match:
        title = title_match.group(1).strip()
        if title == 'Miss' or title == 'Mlle':
            title = 'Ms'
        elif title == 'Mme':
            title = 'Mrs'
        
    if maiden_name_match:
        content_inside_parentheses = maiden_name_match.group(1)
        maiden_name = content_inside_parentheses.split()[-1]
    else:
        maiden_name = last_name
    

    return title, maiden_name, last_name

In [8]:
def drop_full_name(dataset):
    dataset = dataset.drop('Name', axis=1)
    return dataset

## Calling Preprocessing Functions

In [9]:
dataset = drop_ticket(dataset)

In [10]:
dataset = get_all_family_members(dataset)

In [11]:
dataset = clean_nans(dataset)

In [12]:
dataset[['Title', 'MaidenName', 'LastName']] = dataset['Name'].apply(lambda x: pd.Series(transform_name(x)))

In [13]:
dataset = drop_full_name(dataset)

# Save New Dataset

In [14]:
dataset.head()

,PassengerId,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Cabin,Embarked,FamilyMembers,Title,MaidenName,LastName
0,1,0,3,male,22.0,1,0,7.2500,Z,S,1,Mr,Braund,Braund
1,2,1,1,female,38.0,1,0,71.2833,C,C,1,Mrs,Thayer,Cumings
2,3,1,3,female,26.0,0,0,7.9250,Z,S,0,Ms,Heikkinen,Heikkinen
3,4,1,1,female,35.0,1,0,53.1000,C,S,1,Mrs,Peel,Futrelle
4,5,0,3,male,35.0,0,0,8.0500,Z,S,0,Mr,Allen,Allen


In [16]:
dataset.to_csv('data/train_preprocessed.csv', index=False)